In [13]:
import pymysql
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

#cosine 유사도 함수
def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

def Loadfooddataeset (input_food_list):
    #data base 접근
    conn = pymysql.connect(
        host='foodrm.cgnnqocprf5c.us-east-1.rds.amazonaws.com',
        user = 'admin',
        password = '1q2w3e4r',
        db = 'food_info',
        charset = 'utf8'
    )
    cur =conn.cursor()

    Types = set()
    food_list = list()

    # input_food들의 type 추출  ex) 한식, 중식 ...
    for i_food in input_food_list :    
        query = "select type from food where f_name = (%s)"
        cur.execute(query,(i_food))
        rows = cur.fetchall()
        for row in rows :
            tmp = str(row).split("'")[1]
            Types.append(tmp)

    # type이 같은 food_name, feature 추출 
    for Type in Types :
        query = "select * from ingredient left outer join food on ingredient.f_name = food.f_name where food.type = (%s)"
        cur.execute(query,(Type))
        rows = cur.fetchall()
        for row in rows :
            food_list.append(row[:-2])

    
    #index = 음식이름, dataframe = feature, 음식같의 vector
        
    df = DataFrame(np.array(food_list).transpose()[1:].transpose(), index = np.array(food_list).transpose()[0])

    df1 = df.drop(index = input_food_list)  # input 음식들을 추천에서 제외
    conn.close()

    return df1

def Loadfoodfeature(input_food_list) :
    conn = pymysql.connect(
        host='foodrm.cgnnqocprf5c.us-east-1.rds.amazonaws.com',
        user = 'admin',
        password = '1q2w3e4r',
        db = 'food_info',
        charset = 'utf8'
    )
    cur = conn.cursor()
    #input_food_feature 초기화
    input_food_feature = np.zeros(14)
    for food in input_food_list :
        query = "select * from ingredient where f_name = (%s)"
        cur.execute(query,(food))
        rows = cur.fetchall()
        for row in rows :
            input_food_feature += np.array(row[1:])
    input_food_feature = input_food_feature / len(input_food_list)
    conn.close()

    return input_food_feature


input_list = ['감자탕','김치찌개'] #수정사항

df = Loadfooddataeset(input_list)
food_feature = Loadfoodfeature(input_list)


# { 음식이름 : 코사인 유사도 } 사전생성
f_dict = dict()
indexes = df.index
for index in indexes :
    f_dict.update({index : cosine_sim(food_feature, np.array(df.loc[index], dtype = np.float64))})

## print(f_dict)

#코사인 유사도 value 기준으로  reverse sort
rank = sorted(f_dict.items(), key = (lambda x:x[1]), reverse =True)


#상위 3개 list 반환 
rmfood = np.array(rank[:6]).transpose()[0].tolist()

print (rmfood)

['보쌈', '제육볶음', '잡채', '된장찌개', '비빔밥', '돼지국밥']


In [6]:
for r in rank :
    print(r)

('김치찌개', 0.9036961141150639)
('불고기', 0.8660254037844387)
('보쌈', 0.8333333333333335)
('제육볶음', 0.8333333333333335)
('비빔밥', 0.8249579113843055)
('마파두부', 0.8249579113843055)
('잡채', 0.7745966692414834)
('마라샹궈', 0.7216878364870323)
('샤오롱바오', 0.7216878364870323)
('짜장면', 0.7216878364870323)
('된장찌개', 0.6666666666666667)
('족발', 0.6123724356957945)
('꿔바로우', 0.5000000000000001)
('탕수육', 0.5000000000000001)
('육회', 0.43301270189221935)
('짬뽕', 0.43301270189221935)
('돼지국밥', 0.40824829046386296)
('오징어볶음', 0.40824829046386296)
('깐풍기', 0.40824829046386296)
('칠리새우', 0.0)


In [3]:
print(len(rank))

31


In [11]:
print(type(Type))

NameError: name 'Type' is not defined